# 课堂随记

## Books

### 3.SICP | 1.Artificial Intelligence A Modern Approach | 2.Programming Pearls | 4.Fluent Python | W

### Build a Modern Computer from First principles: from nerd to Tetris

# NIPS ACL ICML ICLR AAAI

1.Words -> 2.Entity -> 3.Syntax -> 4.Semantic -> 5.Dialogue -> 6.Reading Comprehension -> 7.Generation -> 8.Representation + Policy -> 1.Words

目前对话系统主要基于搜索，核心点是语义匹配

Reading Comprehension 也可以做自动摘要

Generation 可以做财务报表自动生成

AI ：  
1. Representation 表征  
2. policy 计算

why NLP？
* Information Chaos  
* Unstructured (单词和句子如何表征现在仍然解决不了，但作了很好的尝试:word2vec)
* Discrete（数据是不连续的）
* Unconventional(语言是不常规的，图像则不会）
* OOV（out of vocabulary)

---

## 数组压平实现

### itertools

In [118]:
import itertools
a = ["a", "b", ["c", ["d", "e"]]]
out = list(itertools.chain.from_iterable(a))
out

['a', 'b', 'c', ['d', 'e']]

### 函数递归实现

In [89]:
from icecream import ic
def flatten_list(l):
    ic(l)
    output = []
    for i in l:
        if type(i) == list:
            output+=flatten_list(i)  # 一般遇到list的情况下递归
        else:
            output.append(i)  #递归到最底层时的条件
    ic(output)
    return output

In [90]:
flatten_list([1, [2, 3,[4, 5]]])

ic| l: [1, [2, 3, [4, 5]]]
ic| l: [2, 3, [4, 5]]
ic| l: [4, 5]
ic| output: [4, 5]
ic| output: [2, 3, 4, 5]
ic| output: [1, 2, 3, 4, 5]


[1, 2, 3, 4, 5]

### lambda表达式递归实现

In [93]:
flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) in [tuple, list] else [x] 

In [94]:
flatten(a)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [100]:
flat = lambda L: sum(map(flat, L), []) if isinstance(L, list) else [L]

In [101]:
flat(a)

[1, 2, 3, 4, 5, 6, (7, 8), 9]

---

## Dynamic Programming for cutting problem
* Rob Cutting Problem
* Edit-Distance Problem
* Key Characteristics for Dynamic Programming
* The Travel Salesman Problem

In [26]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]

In [27]:
from collections import defaultdict

In [28]:
price = defaultdict(int)

In [29]:
for i, p in enumerate(original_price):
    price[i+1] = p

In [30]:
assert price[1] == 1

In [31]:
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)]
#         [r(n) + r(0)] + [r(i) + r(n-i) for i in range(1, n)]  这样递归会没有出口
    )

In [32]:
price[132]  # 说明肯定要拆分

0

In [33]:
r(8)

22

In [34]:
# !mv 图片路径 .  

In [35]:
solution = {}

In [36]:
from functools import lru_cache

In [37]:
@lru_cache(maxsize=2**10)
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key = lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    return max_price

In [38]:
solution = {}

In [39]:
r(100)

300

In [40]:
def fun_1(n):

    for i in range(n):
        print(n)


In [41]:
import time

In [42]:
def func_slow(n):
    start = time.time()
    for i in range(n):
        time.sleep(0.2)
        print(n)
    print('used time: {}'.format(time.time() -  start))

In [43]:
func_slow(5)

5
5
5
5
5
used time: 1.0200927257537842


In [44]:
def call_time(fun_1, arg): # 脚手架程序
    start = time.time()
    fun_1(arg)
    print('used time: {}'.format(time.time() -  start))

In [45]:
call_time(fun_1, 10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0056610107421875


In [46]:
def get_call_time(func):
    def _inner(arg):
        start = time.time()
        result = func(arg)
        print('used time: {}'.format(time.time() - start))
        return result
    return _inner

In [47]:
func_1_could_get_calltime  = get_call_time(fun_1)

In [48]:
func_1_could_get_calltime(10)

10
10
10
10
10
10
10
10
10
10
used time: 0.004461050033569336


### 所以就可以用以下方法

In [49]:
fun_1 = get_call_time(fun_1)

In [50]:
fun_1(10)

10
10
10
10
10
10
10
10
10
10
used time: 0.005825042724609375


### 为了避免复杂，所以可以用

In [51]:
@get_call_time
def fun_1(n):

    for i in range(n):
        print(n)

In [52]:
fun_1(10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0004889965057373047


### @w a <=> a = w(a)

### 现在再继续改写

In [53]:
function_called_time = defaultdict(int)

def get_call_time(func):
    def _inner(arg):
        global function_called_time
        function_called_time[func.__name__] += 1
        start = time.time()
        result = func(arg)
        print('function call time is: {}'.format(function_called_time[func.__name__]))
        return result
    return _inner

In [54]:
fun_1(10)    # 可用于累计用户访问数

10
10
10
10
10
10
10
10
10
10
used time: 0.0016660690307617188


---

In [55]:
def memo(func):
    cache = {}
#     @wraps(func)
    def _wrap(n): ## ? *args, **kwargs
        if n in cache: result = cache[n]
        else:
            cache[n] = func(n)
            result = cache[n]
        return result
    return _wrap

In [56]:
@memo
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key = lambda x: x[0]
    )
    
    solution[n] = (split_point, n - split_point)
    
    return max_price

In [57]:
r(400)

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
solution[18]

### Dynamic Programming 
不断查表的意思
+ 分析字问题的重复性
+ 子问题要进行存储
+ Solution要进行解析

In [ ]:
def not_cut(spilt): return spilt == 0

def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    
    if not_cut(left): return [right]
    
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [58]:
parse_solution(18, solution)

NameError: name 'parse_solution' is not defined

### 左边等于0就停止，然后返回右边，[2] + parse_solution(16)

---

# Edit Distance
* Insertion
* Deletion
* Substitution

In [59]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        
    ]
    
    return min(
        edit_distance(string1[:-1], string2) + 1,
        edit_distance(string1, string2[:-1]) + 1,
        edit_distance(string1[:-1], string2[:-1]) + (2 if tail_s1 != tail_s2 else 0)
    )

In [60]:
edit_distance('我今天确实不太想吃饭', '我今天确实不太像吃饭')

2

In [61]:
from icecream import ic

In [66]:
th = 0
# @lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    global th 
        
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        # 先运行第一步，string1先递归到最后，
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 2, 'SUB {} => {}'.format(tail_s1, tail_s2))
    
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    th += 1
    
    ic(string1, string2, min_distance, operation, th)
    
    solution[(string1, string2)] = operation
    
    return min_distance

In [67]:
edit_distance('100', '110')

ic| string1: '1', string2: '1', min_distance: 0, operation: '', th: 1
ic| string1: '1'
    string2: '11'
    min_distance: 1
    operation: 'ADD 1'
    th: 2
ic| string1: '1'
    string2: '110'
    min_distance: 2
    operation: 'ADD 0'
    th: 3
ic| string1: '1', string2: '1', min_distance: 0, operation: '', th: 4
ic| string1: '1'
    string2: '11'
    min_distance: 1
    operation: 'ADD 1'
    th: 5
ic| string1: '1', string2: '1', min_distance: 0, operation: '', th: 6
ic| string1: '10'
    string2: '1'
    min_distance: 1
    operation: 'DEL 0'
    th: 7
ic| string1: '1', string2: '1', min_distance: 0, operation: '', th: 8
ic| string1: '10'
    string2: '11'
    min_distance: 2
    operation: 'DEL 0'
    th: 9
ic| string1: '1', string2: '1', min_distance: 0, operation: '', th: 10
ic| string1: '1'
    string2: '11'
    min_distance: 1
    operation: 'ADD 1'
    th: 11
ic| string1: '10'
    string2: '110'
    min_distance: 1
    operation: ''
    th: 12
ic| string1: '1', string2: '1', 

2

In [174]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

In [69]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

In [69]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73:

In [69]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10),
 21: (1, 20),
 22: (2, 20),
 23: (3, 20),
 24: (2, 22),
 25: (2, 23),
 26: (6, 20),
 27: (1, 26),
 28: (2, 26),
 29: (3, 26),
 30: (10, 20),
 31: (1, 30),
 32: (2, 30),
 33: (3, 30),
 34: (2, 32),
 35: (2, 33),
 36: (6, 30),
 37: (1, 36),
 38: (2, 36),
 39: (3, 36),
 40: (10, 30),
 41: (1, 40),
 42: (2, 40),
 43: (3, 40),
 44: (2, 42),
 45: (2, 43),
 46: (6, 40),
 47: (1, 46),
 48: (2, 46),
 49: (3, 46),
 50: (10, 40),
 51: (1, 50),
 52: (2, 50),
 53: (3, 50),
 54: (2, 52),
 55: (2, 53),
 56: (6, 50),
 57: (1, 56),
 58: (2, 56),
 59: (3, 56),
 60: (10, 50),
 61: (1, 60),
 62: (2, 60),
 63: (3, 60),
 64: (2, 62),
 65: (2, 63),
 66: (6, 60),
 67: (1, 66),
 68: (2, 66),
 69: (3, 66),
 70: (10, 60),
 71: (1, 70),
 72: (2, 70),
 73: